In [1]:
from sympy import MatrixSymbol, Matrix, Identity

import sympy as sym
import pandas as pd

sym.init_printing()
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import control
import random

# IDENTIFICACIÓN DE SISTEMAS

Profesor: Jairo Alberto Cuéllar Guarnizo  
Programa: Ingeniería en Automatización y Control

## Optimización Convexa



# Introducción a la Optimización

## El problema de la optimización

La optimización se refiere a los parámetros desconocidos como variable de decisión ($\theta$), frente a una función objetivo que depende de los datos medidos y usualmente llamada Y. Por ejemplo para Mínimos cuadrados, el problema de optimización es minimizar el error existente entre el vector Y y el modelo de predicción $M(\theta)$. La función a minimizar sería:

$$f(\theta):=|Y - M(\theta)|^2$$

Para efectos de los ejercicios simplemente se llamará "x" a la variable de decisión desconocida en el problema de optimización.

$$\min_{x \epsilon D} f(x)$$

Donde D es un espacio vectorial $R^n$.